In [ ]:
import gymnasium as gym
import numpy as np


n_states = 16
n_actions = 4
goal_state = 15
hole = [6,8,12,13]

Q_table = np.zeros((n_states, n_actions))
env = gym.make(
    'FrozenLake-v1',
    desc=None,
    map_name="4x4",
    is_slippery=False,
)


def get_next_state(state, action):
    row, col = divmod(state, 4)

    if action == 0 and col > 0:
        col -= 1
    elif action == 1 and col < 3:
        col += 1
    elif action == 2 and row > 0:
        row -= 1
    elif action == 3 and row < 3:
        row += 1

    return row * 4 + col
learning_rate = 0.8
discount_factor = 0.95
ep = 1000
total_reward = 0
eps_start= 0.9
eps_end= 0.01
eps_decay= 0.99

for ep in range(ep):
    current_state, info = env.reset()
    done = False

    while not done:
        epsilon = max(eps_end, eps_start * (eps_decay)**(ep))
        if np.random.rand() < epsilon:
            action = np.random.randint(0, n_actions)
        else:
            action = np.argmax(Q_table[current_state])

        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        if next_state == goal_state:
            reward = 10
        elif next_state in hole:
            reward = -1
        else:
            reward = 0

        total_reward += reward

        Q_table[current_state, action] += learning_rate * (
            reward + discount_factor * np.max(Q_table[next_state])
            - Q_table[current_state, action]
        )

        current_state = next_state
np.save("q_table.npy", Q_table)
print(f"Total reward after training: {total_reward}")
print("Training finished.")
print("Q-table:")
print(Q_table)
optimal_actions = np.argmax(Q_table, axis=1)
print(optimal_actions.reshape((4,4)))